# SQL Básico

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [1]:
%defaultDatasource jdbc:h2:mem:db

## Exemplo dos Táxis

Baseado no exemplo criado por prof. Geovane Cayres Magalhães http://www.ic.unicamp.br/~geovane/mo410-091/caso.html

## Conjunto de tabelas para consultas básicas

In [2]:
DROP TABLE IF EXISTS Taxi;
DROP TABLE IF EXISTS Cliente;
DROP TABLE IF EXISTS Corrida;

In [3]:
CREATE TABLE Taxi (
  Placa VARCHAR(7) NOT NULL,
  Marca VARCHAR(30) NOT NULL,
  Modelo VARCHAR(30) NOT NULL,
  AnoFab INTEGER,
  Licenca VARCHAR(9),
  PRIMARY KEY(Placa)
);

CREATE TABLE Cliente (
  CliId VARCHAR(4) NOT NULL,
  Nome VARCHAR(80) NOT NULL,
  CPF VARCHAR(14) NOT NULL,
  PRIMARY KEY(CliId)
);

CREATE TABLE Corrida (
  CliId VARCHAR(4) NOT NULL,
  Placa VARCHAR(7) NOT NULL,
  DataPedido DATE NOT NULL,
  PRIMARY KEY(CliId, Placa, DataPedido),
  FOREIGN KEY(CliId)
    REFERENCES Cliente(CliId)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(Placa)
    REFERENCES Taxi(Placa)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
);

In [4]:
INSERT INTO Cliente VALUES ('1532', 'Asdrúbal', '448.754.253-65');
INSERT INTO Cliente VALUES ('1755', 'Doriana', '567.387.387-44');
INSERT INTO Cliente VALUES ('1780', 'Quincas', '546.373.762-02');
INSERT INTO Cliente VALUES ('1888', 'Melissa', '123.456.789-10');

INSERT INTO Taxi VALUES ('DAE6534', 'Ford', 'Fiesta', 1999, 'MN572345');
INSERT INTO Taxi VALUES ('DKL4598', 'Wolkswagen', 'Gol', 2001, 'AU876543');
INSERT INTO Taxi VALUES ('DKL7878', 'Ford', 'Fiesta', 2001, 'OP102938');
INSERT INTO Taxi VALUES ('JDM8776', 'Wolkswagen', 'Santana', 2002, 'QM365923');
INSERT INTO Taxi VALUES ('JJM3692', 'Chevrolet', 'Corsa', 1999, 'UU335577');

INSERT INTO Corrida VALUES ('1755', 'DAE6534', '2003-02-15');
INSERT INTO Corrida VALUES ('1780', 'JDM8776', '2003-02-18');
INSERT INTO Corrida VALUES ('1755', 'DKL7878', '2003-02-16');
INSERT INTO Corrida VALUES ('1780', 'DKL4598', '2003-02-17');
INSERT INTO Corrida VALUES ('1532', 'DKL4598', '2003-02-18');
INSERT INTO Corrida VALUES ('1780', 'DAE6534', '2003-02-16');
INSERT INTO Corrida VALUES ('1755', 'DAE6534', '2003-02-18');
INSERT INTO Corrida VALUES ('1532', 'DKL4598', '2003-02-20');

# SELECT Básico

### Todos os Táxis com todos os atributos

In [5]:
SELECT * FROM Taxi;

### Marca e Modelo dos Taxis disponíveis

In [6]:
SELECT Marca, Modelo FROM Taxi;

### Taxis fabricados depois do ano 2000

In [7]:
SELECT * FROM Taxi WHERE AnoFab > 2000;

### Placas que comecem com DK

In [8]:
SELECT * FROM Taxi WHERE placa LIKE 'DK%';

### Placas com '7' na penultima posicao

In [9]:
SELECT * FROM Taxi WHERE placa LIKE '%7_';

# Produto Cartesiano

### Produto Cartesiano entre Clientes e Corridas

In [10]:
SELECT Cliente.CliId, Cliente.Nome, Corrida.Placa, Corrida.DataPedido
FROM Cliente, Corrida

# JOIN Implícito

### Cientes e as respectivas corridas - Join Implícito

In [11]:
SELECT Cliente.CliId, Cliente.Nome, Corrida.Placa, Corrida.DataPedido
FROM Cliente, Corrida
WHERE Cliente.CliId = Corrida.CliId

### Modelo de taxi para cada corrida

In [12]:
SELECT Corrida.DataPedido, Corrida.Placa, Taxi.Modelo
       FROM Corrida, Taxi
       WHERE Corrida.Placa = Taxi.Placa;

### Modelos de Táxi por Cliente (estágio 1)

In [13]:
SELECT Cliente.Nome, Corrida.DataPedido, Corrida.Placa, Taxi.Modelo
       FROM Cliente, Corrida, Taxi
       WHERE Cliente.CliId = Corrida.CliId AND Corrida.Placa = Taxi.Placa;

### Modelos de Táxi por Cliente (estágio 2)

In [14]:
SELECT DISTINCT Cliente.Nome, Taxi.Modelo
       FROM Cliente, Corrida, Taxi
       WHERE Cliente.CliId = Corrida.CliId AND Corrida.Placa = Taxi.Placa;

## ORDER BY

### Nome dos clientes ordenado alfabeticamente

In [8]:
SELECT Nome
       FROM Cliente
       ORDER BY Nome DESC;

### Modelos de taxi e os clientes que os tomaram

Ordena por Modelo, mas não por Cliente:

In [16]:
SELECT DISTINCT Cl.Nome, T.Modelo
       FROM Cliente Cl, Corrida Co, Taxi T
       WHERE Cl.CliId = Co.CliId AND
             Co.Placa = T.Placa
       ORDER BY T.Modelo;

Ordena por Modelo e, para cada Modelo, por Cliente:

In [17]:
SELECT DISTINCT Cl.Nome, T.Modelo
       FROM Cliente Cl, Corrida Co, Taxi T
       WHERE Cl.CliId = Co.CliId AND
             Co.Placa = T.Placa
       ORDER BY T.Modelo, Cl.Nome;

### Cliente e os modelos de taxi tomados
Ordena por Cliente, mas não por Modelo:

In [18]:
SELECT DISTINCT Cl.Nome, T.Modelo
       FROM Cliente Cl, Corrida Co, Taxi T
       WHERE Cl.CliId = Co.CliId AND
             Co.Placa = T.Placa
       ORDER BY Cl.Nome;

Ordena por Cliente e, para cada Cliente, por Modelo

In [19]:
SELECT DISTINCT Cl.Nome, T.Modelo
       FROM Cliente Cl, Corrida Co, Taxi T
       WHERE Cl.CliId = Co.CliId AND
             Co.Placa = T.Placa
       ORDER BY Cl.Nome, T.Modelo;

## JOIN explícito

### Clientes e respectivas corridas (para clientes que fizeram corrida)

In [20]:
SELECT Cl.CliId, Cl.Nome,
       Co.Placa, Co.DataPedido
       FROM Cliente Cl JOIN Corrida Co
            ON Cl.CliId = Co.CliId;

### Taxis e respectivas corridas (para taxis que fizeram corrida)

In [21]:
SELECT Tx.placa, Co.cliid
       FROM Taxi Tx JOIN Corrida Co
            ON Tx.placa = Co.placa;

## NATURAL JOIN

In [22]:
SELECT Tx.placa, Co.cliid
       FROM Taxi Tx
            NATURAL JOIN Corrida Co;

## LEFT JOIN

### Clientes e respectivas corridas (para todos os clientes)

In [23]:
SELECT Cl.CliId, Cl.Nome,
       Co.Placa, Co.DataPedido
       FROM Cliente Cl LEFT JOIN Corrida Co
            ON Cl.CliId = Co.CliId;

### Taxis e respectivas corridas (para todos os taxis)

In [24]:
SELECT Tx.placa, Co.cliid
       FROM Taxi Tx LEFT JOIN Corrida Co
            ON Tx.placa = Co.placa;

## RIGHT JOIN

### Corridas e respectivos clientes (para todos os clientes)

In [25]:
SELECT Co.placa, Cl.nome
       FROM Corrida Co RIGHT JOIN Cliente Cl
            ON Co.cliid = Cl.cliid;

# GROUP BY

### Modelos de Táxi cadastrados

In [9]:
SELECT T.Modelo
       FROM Taxi T
       GROUP BY T.Modelo;

### Total de Táxis por Modelo

Modelos de Táxi cadastrados e quantos táxis há em cada modelo:

In [10]:
SELECT T.Modelo, COUNT(*)
       FROM Taxi T
       GROUP BY T.Modelo;

## Agregando em mais de um nível

### Modelos de taxi tomados por cada cliente (agrupando por Cliente e por Modelo)

In [18]:
SELECT Cl.Nome, T.Modelo
       FROM Cliente Cl, Corrida Co, Taxi T
       WHERE Cl.CliId = Co.CliId AND
             Co.Placa = T.Placa;

In [29]:
SELECT DISTINCT Cl.Nome, T.Modelo
       FROM Cliente Cl, Corrida Co, Taxi T
       WHERE Cl.CliId = Co.CliId AND
             Co.Placa = T.Placa;

In [30]:
SELECT Cl.Nome, T.Modelo
       FROM Cliente Cl, Corrida Co, Taxi T
       WHERE Cl.CliId = Co.CliId AND
             Co.Placa = T.Placa
       GROUP BY Cl.Nome, T.Modelo;

In [11]:
SELECT Cl.Nome, T.Modelo, COUNT(*)
       FROM Cliente Cl, Corrida Co, Taxi T 
       WHERE Cl.CliId = Co.CliId AND 
             Co.Placa = T.Placa
       GROUP BY Cl.Nome, T.Modelo;

## Contando Itens Distintos

### Quantos clientes distintos tomaram cada modelo de táxi

In [13]:
SELECT T.Modelo, Cl.Nome, COUNT(*)
       FROM Cliente Cl, Corrida Co, Taxi T 
       WHERE Cl.CliId = Co.CliId AND 
             Co.Placa = T.Placa
       GROUP BY T.Modelo, Cl.Nome;

#### Sem distinguir clientes

In [16]:
SELECT T.Modelo, COUNT(*)
       FROM Taxi T, Corrida Co
       WHERE Co.Placa = T.Placa
       GROUP BY T.Modelo;

#### Com distinção de clientes

In [17]:
SELECT T.Modelo, COUNT(DISTINCT Co.CliId)
       FROM Taxi T, Corrida Co
       WHERE Co.Placa = T.Placa
       GROUP BY T.Modelo;

## Funções de agregação sem agrupamento

### Média de ano de fabricação para todos os Táxis

Quando é usada função de agregação sem especificação de agrupamento, toda a tabela é agregada como se fosse um único grupo.

In [32]:
SELECT Modelo, AnoFab FROM Taxi;

In [33]:
SELECT Modelo, MIN(AnoFab), MAX(AnoFab), AVG(AnoFab)
       FROM Taxi
       GROUP BY Modelo;

In [34]:
-- Maior ano de fabricação de toda a tabela
SELECT MAX(AnoFab) FROM Taxi;

2002

## Campos mostrados devem corresponder a agregação

Exemplo sem agregação:

In [35]:
SELECT Modelo, AnoFab
       FROM Taxi;

### Modelo e Ano de Fabricação

Este exemplo com agregação produzirá um erro pois `AnoFab` (ano de fabricação) não foi agregado, portanto, pode haver mais de um ano de fabricação para o mesmo modelo.

In [36]:
SELECT Modelo, AnoFab
       FROM Taxi
       GROUP BY Modelo;

org.h2.jdbc.JdbcSQLException:  Column "ANOFAB" must be in the GROUP BY list; SQL statement

### Modelo e Maior Ano de Fabricação
#### Corrigindo com função de agregação

Este exemplo escolhe o maior ano para cada Modelo agregado.

In [37]:
SELECT Modelo, MAX(AnoFab)
       FROM Taxi
       GROUP BY Modelo;

# HAVING

### Número de Táxis por Modelo somente para os Táxis com ano de fabricação abaixo de 2000

Exemplo com `WHERE` em que a condição é aplicada antes de se agregar:

In [38]:
-- Todos os Táxis
SELECT Modelo, AnoFab
       FROM Taxi;

In [39]:
SELECT Modelo, AnoFab
       FROM Taxi
       WHERE AnoFab < 2000;

In [40]:
-- Selecionados e Agregados
SELECT Modelo, COUNT(*) NUM
       FROM Taxi
       WHERE AnoFab < 2000
       GROUP BY Modelo;

### Modelos de Táxi que têm mais de um Táxi do respectivo modelo

Exemplo com `HAVING` em que a condição é aplicada após se agregar:

In [41]:
-- Agrupamento
SELECT Modelo, COUNT(*) NUM
       FROM Taxi
       GROUP BY Modelo;

In [42]:
-- Agrupamento com HAVING
SELECT Modelo, COUNT(*) NUM
       FROM Taxi
       GROUP BY Modelo
       HAVING NUM > 1;

## `HAVING` apenas para campos agregados 

### Modelos daqueles Táxis com ano de fabricação acima de 2000

A seleção a seguir produzirá um erro pois está se aplicando uma condição `HAVING` para um campo não agregado. Deveria ter sido usado `WHERE` como foi feito anteriormente.

In [43]:
SELECT Modelo, COUNT(*) NUM
       FROM Taxi
       GROUP BY Modelo
       HAVING AnoFab > 2000;

org.h2.jdbc.JdbcSQLException:  Column "ANOFAB" must be in the GROUP BY list; SQL statement

## Exemplo de `HAVING` com funções de agregação

### Modelos de Táxi cujo menor Ano de Fabricação seja após 2000

In [44]:
-- sem agregacao (nao eh possível resolver a questao)
SELECT Modelo, AnoFab
       FROM Taxi
       WHERE AnoFab < 2000;

In [45]:
-- testando após a agregação
SELECT Modelo, MAX(AnoFab) MaiorAno
       FROM Taxi
       GROUP BY Modelo
       HAVING MaiorAno < 2000;

# VIEW

### Total de Táxis por Modelo

Agrupamento sem o uso de `VIEW`:

In [46]:
SELECT Modelo, COUNT(*) Numero_Taxis
       FROM taxi
       GROUP BY Modelo;

### Tabela de Total de Táxis por Modelo

Transformando o agrupamento na tabela `Contagem_Modelo` com o `VIEW`:

In [47]:
CREATE VIEW Contagem_Modelo AS
SELECT Modelo, COUNT(*) Numero_Taxis
       FROM taxi
       GROUP BY Modelo;

SELECT * FROM Contagem_Modelo;

Usando a tabela criada com a `VIEW`:

In [48]:
SELECT MAX(Numero_Taxis) FROM Contagem_Modelo;

2

## VIEW se auto-atualiza

A view é um recorte dinâmico, ela se auto-atualiza quando a tabela original é modificada.

In [49]:
INSERT INTO Taxi VALUES ('KMN3412', 'Chevrolet', 'Corsa', 2001, 'QJ572345');
SELECT * FROM Contagem_Modelo;